In [8]:
### Install dependencies ###
!pip install keras numpy sklearn > /dev/null 2>&1

In [9]:
### Imports ###

# Keras
import keras.utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
# Processing
import numpy as numpy
# Used to encode labels like the monster classes to one-hot vectors
from sklearn.preprocessing import LabelEncoder
# Static typing ftw
from typing import List, cast

In [10]:
### Data processing ###

def labels_to_numbers(labels: List[str]) -> List[int]:
    """
    Transforms:
      ['Ghoul', 'Goblin', 'Ghoul', 'Ghoul', 'Ghost',
    to:
      [1, 2, 1, 1, 0,
    """
    return LabelEncoder().fit(labels).transform(labels)


def load_features(file_name: str) -> List:
    """
    Loads the features we are using for inferring the target.
    Those features are: bone_length  rotting_flesh  hair_length  has_soul color

    The csv files look like this:

    id  bone_length  rotting_flesh  hair_length  has_soul  color    type
    0     0.354512       0.350839     0.465761  0.781142  clear   Ghoul
    1     0.575560       0.425868     0.531401  0.439899  green  Goblin
    2     0.467875       0.354330     0.811616  0.791225  black   Ghoul
    """
    path = 'data/' + file_name
    numeric_features = numpy.genfromtxt(path,
                                        skip_header=1,
                                        delimiter=",",
                                        usecols=(1, 2, 3, 4))
    colors = numpy.genfromtxt(path,
                              skip_header=1,
                              delimiter=",",
                              usecols=5,
                              dtype=str)
    accumulator = []
    for row, color_as_number in zip(numeric_features, labels_to_numbers(colors)):
        accumulator.extend([numpy.append(row, color_as_number)])
    return numpy.asarray(accumulator)

def load_targets(file_name: str) -> List:
  """
  Loads the labels we are using for inferring the target ('Ghost', 'Goblin', 'Ghoul')
  and transforms them into a one-hot vector.
  So basically this:
    ['Ghoul', 'Goblin', 'Ghoul', 'Ghoul', 'Ghost', .....
  becomes:
    [[ 0.,  1.,  0.],
     [ 0.,  0.,  1.],
     [ 0.,  1.,  0.],
  """
  labels = numpy.genfromtxt('data/' + file_name,
                            skip_header=1,
                            delimiter=",",
                            usecols=6,
                            dtype=str)
  return keras.utils.to_categorical(labels_to_numbers(labels))

In [11]:
### Load the data ###

# We have 371 records for training but no test data.
# So let's just use the first 300 records for training and the last 71 for 
SLICE_AT = 300

features = load_features('train.csv')
targets = load_targets('train.csv')
x_train = features[0:SLICE_AT]
y_train = targets[0:SLICE_AT]
x_test = features[SLICE_AT:]
y_test = targets[SLICE_AT:]
x_for_prediction = load_features('test.csv')
y_for_prediction = 'TODO'

In [12]:
### Build the models ###

FEATURES = ['bone_length', 'rotting_flesh', 'hair_length', 'has_soul', 'color']
INPUT_DIM = len(FEATURES)
VALID_TYPES = ['Ghost', 'Goblin', 'Ghoul']
OUTPUT_DIM = len(VALID_TYPES)
scores_to_models = {}

no_hidden_layers = Sequential()

# Basic no_hidden_layers inspired from:
# https://keras.io/getting-started/sequential-no_hidden_layers-guide/
# section: "Multilayer Perceptron (MLP) for multi-class softmax classification"
#
# Play around with:
# * dropout
# * learning algorithm and its parameters
# * activation
# * learning rate
no_hidden_layers.add(Dense(10, activation='relu', input_dim=INPUT_DIM))
no_hidden_layers.add(Dense(OUTPUT_DIM, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
no_hidden_layers.compile(loss='categorical_crossentropy',
                         optimizer=sgd,
                         metrics=['accuracy'])

no_hidden_layers.fit(x_train, y_train, epochs=20, batch_size=128)
score = no_hidden_layers.evaluate(x_test, y_test, batch_size=128)

scores_to_models['no_hidden_layers'] = score

Epoch 1/20
128/300 [===========>..................] - ETA: 0s - loss: 1.6282 - acc: 0.2500

300/300 [==============================] - 0s - loss: 1.4740 - acc: 0.3233     


Epoch 2/20
128/300 [===========>..................] - ETA: 0s - loss: 1.3456 - acc: 0.3125

300/300 [==============================] - 0s - loss: 1.2784 - acc: 0.3267     


Epoch 3/20
128/300 [===========>..................] - ETA: 0s - loss: 1.1573 - acc: 0.3359

...


Epoch 19/20
128/300 [===========>..................] - ETA: 0s - loss: 1.1114 - acc: 0.3125

300/300 [==============================] - 0s - loss: 1.1091 - acc: 0.3100     


Epoch 20/20
128/300 [===========>..................] - ETA: 0s - loss: 1.1018 - acc: 0.2734

300/300 [==============================] - 0s - loss: 1.1082 - acc: 0.3033     

71/71 [==============================] - 0s

In [12]:
for k, v in scores_to_models.items():
    print("Model " + k + ":")
    print("  Loss: " + v[0])
    print("  Accuracy: " + v[1])

AttributeError: 'dict' object has no attribute 'iteritems'